[View in Colaboratory](https://colab.research.google.com/github/khanyofficial/inkers1/blob/master/3A.ipynb)

In [2]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda, Conv1D, SeparableConv2D, Dropout
from keras.layers.merge import concatenate
from keras.utils import np_utils
import tensorflow as tf
batch_size = 128
num_classes = 10
epochs = 20


def space_to_depth_x2(l ):
    return tf.space_to_depth(l, block_size=7)

# the data, split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
# Hyperparameters
batch_size = 16
num_classes = 10
epochs = 10
l = 10
num_filter = 20

# Load CIFAR10 Data
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]
input = Input(shape=(img_height, img_width, channel,))

layer1 = Conv2D(16, (5,5), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
layer1 = BatchNormalization(name='norm_1')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)
layer1 = Conv2D(16, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

skip_conn = layer1
print (layer1.shape, skip_conn.shape)

layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)
layer1 = SeparableConv2D(16, (3, 3), strides=(1, 1), padding='same', name='conv_3', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_3')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)
layer1 = Conv2D(16, (3, 3), strides=(1, 1), padding='same', name='conv_4', use_bias=False)(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)


skip_conn = Conv2D(16, (3,3), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_conn)
skip_conn = BatchNormalization(name='norm_21')(skip_conn)
skip_conn = LeakyReLU(alpha=0.1)(skip_conn)
skip_conn = MaxPooling2D(pool_size=(2, 2))(skip_conn)
print (layer1.shape, skip_conn.shape)
skip_conn = Lambda(space_to_depth_x2)(skip_conn)
print (layer1.shape, skip_conn.shape)

layer1 = concatenate([layer1, skip_conn])
layer1 = LeakyReLU(alpha=0.1)(layer1)
layer1 = (Flatten())(layer1)
#layer1 = Dropout(0.2)(layer1)
output = Dense(num_classes, activation='softmax')(layer1)
model = Model(inputs=[input], outputs=[output])
model.summary()

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print(score)

(?, 14, 14, 16) (?, 14, 14, 16)
(?, 1, 1, 16) (?, 7, 7, 16)
(?, 1, 1, 16) (?, 1, 1, 784)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 28, 28, 16)   400         input_2[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 28, 28, 16)   64          conv_1[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu_7 (LeakyReLU)       (None, 28, 28, 16)   0           norm_1[0][0]                     
____________________